## Ball Picking Challenge

### Part 2: Pick and Place

The aim is to program the robot for the second part of the challenge: pick up the balls and transport them to the corner. To do so, you need to reuse the abilities learnt in week 4; please feel free to reuse the code of those notebooks and exercises.

In [59]:
from packages import initialization
from packages import pioneer3dx as p3dx
import numpy as np
import cv2
import math
p3dx.init()
import matplotlib.pyplot as plt
import time
%matplotlib inline

Found ROS controller /pioneer3dx_29386_ip_172_31_33_91


In [60]:
def move(V_robot,w_robot):
    r = 0.1953 / 2
    L = 0.33
    w_r = (2 * V_robot + L * w_robot) / (2*r)
    w_l = (2 * V_robot - L * w_robot) / (2*r)
    p3dx.move(w_l, w_r)

def color_blob():
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    lower = np.array([110,  50,  50])
    upper = np.array([130, 255, 255])
    mask = cv2.inRange(hsv, lower, upper)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    area = None
    cx = None
    cy = None
    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        M = cv2.moments(c)
        area = M['m00']
        if area > 0:
            cx = int(M['m10']/area)
            cy = int(M['m01']/area)
        else:
            cx = None
            cy = None
    return area, cx, cy

def is_blob_centered():
    area, cx, cy = color_blob()
    if area > 0 and cx >= 70 and cx < 80:
        return True
    else:
        return False

def is_blob_close():
    area, cx, cy = color_blob()
    if area > 0 and cy >= 90:
        return True
    else:
        return False

def red_zone():
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    lower = np.array([0,  80,  80])
    upper = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv, lower, upper)
    lower = np.array([170, 80,  80])
    upper = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower, upper)
    mask = mask1 + mask2
    M = cv2.moments(mask)
    area = int(M['m00'])
    if area > 10000:
        cx = int(M['m10']/area)
        cy = int(M['m01']/area)
    else:
        cx = None
        cy = None
    return area, cx, cy

def is_red_centered():
    area, cx, cy = red_zone()
    if area > 0 and cx >= 70 and cx < 80:
        return True
    else:
        return False

def is_red_close():
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    lower = np.array([0,  80,  80])
    upper = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv, lower, upper)
    lower = np.array([170, 80,  80])
    upper = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower, upper)
    mask = mask1 + mask2
    M = cv2.moments(mask)
    area = int(M['m00'])
    if area > 0:
        cx = int(M['m10']/area)
        cy = int(M['m01']/area)
        if cy >= 85:
            return True
        else:
            return False
    else:
        return False

def ball_is_not_on_red():
    area, cx, cy = color_blob()
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    lower = np.array([0,  80,  80])
    upper = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv, lower, upper)
    lower = np.array([170, 80,  80])
    upper = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower, upper)
    mask = mask1 + mask2
    mask[0:cy, 0:150] = 0
    mask[cy+15:, 0:150] = 0
    M = cv2.moments(mask)
    area = int(M['m00'])
    if area > 0:
        return False
    else:
        return True

count = 0

while True:    
    #open the gripper
    p3dx.gripper(0.05,0.1)
    #tilt camera
    #p3dx.tilt(0.0)
    p3dx.tilt(-0.38)
    #p3dx.tilt(-0.25)
    time.sleep(1)
    
    # find and center the ball
    print('Try to find target')
    while not is_blob_centered():
        p3dx.move(-0.5,0.5)
    p3dx.stop()
    
    if ball_is_not_on_red():
        # close to the ball and center all time
        while not is_blob_close():
            while not is_blob_centered():
                p3dx.move(-0.5,0.5)
            p3dx.move(3.0,3.0)
        p3dx.stop()
        
        #fine motion to the ball
        p3dx.tilt(-0.47)
        time.sleep(1)
        while not is_blob_close():
            p3dx.move(1.0,1.0)
        p3dx.stop()
        time.sleep(1)
        
        # grasping
        p3dx.gripper(0.05,0.02) #close
        time.sleep(1.5)
        p3dx.gripper(-0.05,0.0) #up
        time.sleep(1.5)
        
        #search red zone
        p3dx.tilt(0.0)
        time.sleep(1)
        # center the red area
        while not is_red_centered():
            p3dx.move(-0.5,0.5)
        p3dx.stop()
        
        # close to the red zone
        while not is_red_close():
            p3dx.move(5.0,5.0)
        p3dx.stop()
        time.sleep(1)

        p3dx.gripper(0.05,0.1) #open
        time.sleep(0.5)
        #p3dx.gripper(-0.05,0.1) #up
        #
        p3dx.move(-2.0,-2.0)
        time.sleep(5)
        # rotate 200 degrees
        target = 200*math.pi/180      # target angle in radians
        r = 195.3/2           # wheel radius
        L = 330           # axis length
        initialEncoder = p3dx.leftEncoder
        robotAngle = 0
        while robotAngle < target:
            p3dx.move(2.0,-2.0)
            wheelAngle = p3dx.leftEncoder - initialEncoder
            robotAngle = 2*wheelAngle*r/L
        p3dx.stop()
        count = 0
    
    else:
        p3dx.move(-0.5,0.5)
        time.sleep(2)
        print('lost target')
        count += 1
        if count > 2:
            p3dx.stop()
            break

Try to find target
Try to find target
Try to find target
lost target
Try to find target
lost target
Try to find target
Try to find target
Try to find target
lost target
Try to find target
Try to find target


Exception: The robot needs initialization - has the simulation been restarted?

---
#### Try-a-Bot: an open source guide for robot programming
Developed by:
[![Robotic Intelligence Lab @ UJI](img/logo/robinlab.png "Robotic Intelligence Lab @ UJI")](http://robinlab.uji.es)

Sponsored by:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="http://www.ieee-ras.org"><img src="img/logo/ras.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.cyberbotics.com"><img src="img/logo/cyberbotics.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.theconstructsim.com"><img src="img/logo/theconstruct.png"></a>
</td>
</tr>
</table>

Follow us:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="https://www.facebook.com/RobotProgrammingNetwork"><img src="img/logo/facebook.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="https://www.youtube.com/user/robotprogrammingnet"><img src="img/logo/youtube.png"></a>
</td>
</tr>
</table>